# Lab 2 - oczyszczanie danych

Dane w surowej formie nie zawsze nadają się do trenowania modeli predykcyjnych. Jedną z najczęstszych przyczyn zanieczyszczenia danych są brakujące wartości w poszczególnych atrybutach. Istnieją trzy popularne rozwiązania ,,walki" z wartościami brakującymi:
- usunięcie wierszy z wybrakowanymi wartościami,
- usunięcie całego atrybutu z wartościami wybrakowanymi,
- uzupełnienie wybranych wartości wybraną strategią.

Dobór rozwiązania w dużym stopniu zależy od charakteru zbioru danych oraz wartości wybrakowanych. W najbardziej idealnym scenariuszu należy sprawdzić efekt zapewniany przez wszystkie poznane rozwiązania, lecz w przypadku ograniczonych zasobów obliczeniowych może nie zawsze być to osiągalne. W praktyce można spotkać się z intyicyjnymi podejściami, które polegają na usuwaniu wierszy z brakującymi wartościami w przypadku ich niewielkiej liczebności, np. do ok. 0.1% oryginalnego zbioru danych. Podejście na usunięciu całego atrybutu sprawdza się gdy znaczna częśc wartości atrybutu jest wybrakowana. W pozostałych przypadkach najlepiej powinno sprawdzić się uzupełnianie wartości według obranej strategii. Wśród najbardziej typowych strategii uzupełniania wartości wybrakowanych warto wymienić następujące:
- średnia arytmetyczna lub mediana (dla wartości rzeczywistych),
- dominanta (dla danych kategorialnych).

Wśród popularnych podejść można znaleźć także związane z wytrenowanymi predyktorami:
- kNN: dopasowuje brakującą wartość na podstawie najbliższych sąsiadów pozostałych uzupełnionych wartości,
- regresor: dopasowuje brakującą wartość na podstawie modelu regresyjnego wytrenowanego na podstawie uzupełnionych wartości.

Wśród pozostałych metod warto zwrócić także uwagę na kontekst atrybutu, w którym występuje brakująca wartość. Przykładowo, jeżeli wybrakowana wartość dotyczy atrybutu poziomu leukocytów we krwi, warto sprawdzić jakie są normy, a następnie wylosować na podstawie wybranego rozkładu prawdopodobieństwa (np. gaussowskiego) wartość w przedziale stanowiącym normę populacji.

## Oczyszczanie danych na podstawie funkcjonalności biblioteki pandas

In [17]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)['frame']
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


Metoda *isnull* wywołana na obiekcie klasy Series zwróci serię wartości logicznych odpowiadających temu czy dana wartośc w kolumnie jest wybrakowana.

In [18]:
data['MedInc'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
20635    False
20636    False
20637    False
20638    False
20639    False
Name: MedInc, Length: 20640, dtype: bool

Za pomocą metody *any* wywołanej na powstałej w ten sposób ramce można sprawdzić czy występuje tam przynajmniej jedna wartość prawdziwa. Metoda all umożliwia sprawdzenie czy w ramce występują tylko i wyłącznie wartości prawdziwe.

In [19]:
data['MedInc'].isnull().any()

False

Za pomocą odwołania się do osi kolumn (parametr *axis=0* w metodzie *any*) można z łatwością sprawdzić w których kolumnach występują wartości wybrakowane.

In [20]:
data.isnull().any(axis=0)

MedInc         False
HouseAge       False
AveRooms       False
AveBedrms      False
Population     False
AveOccup       False
Latitude       False
Longitude      False
MedHouseVal    False
dtype: bool

Wiemy, że nasz dataset jest pełny. Spróbujmy zatem usunąć parę losowych wystąpień metodą chybił-trafił:

In [21]:
from random import randint

# minimalny i maksymalny odsetek komorek do usuniecia wartosci
min_percent, max_percent = 0.001, 0.003

# wyznacza pseudolosowo od min_percent do max_percent komórek z ramki danych
cells_to_remove = randint(int(data.size * min_percent), int(data.size * max_percent))

# pseudolosowy wybor indeksow wierszy i kolumn
for _ in range(cells_to_remove):
  row_idx = randint(0, data.shape[0] - 1)  # pseudolosowy indeks wiersza
  col_idx = randint(0, data.shape[1] - 1)  # pseudolosowy indeks kolumny

  # usuniecie pseudolosowo wskazanej komorki
  data.iat[row_idx, col_idx] = None

Oczywiścke istnieje takie prawdopodobieństwo, że "usuniemy" 2 bądź więcej razy tą samą komórkę, natomiast w tym przykładzie niezależy nam na unikalności usuwania.

Sprawdźmy zatem, jak teraz wygląda nasz dataset:

In [22]:
data.isnull().any(axis=0)

MedInc         True
HouseAge       True
AveRooms       True
AveBedrms      True
Population     True
AveOccup       True
Latitude       True
Longitude      True
MedHouseVal    True
dtype: bool

Jak widać udało nam się w każdej kolumnie uzyskać wartości brakujące. Doskonale!

Prześledźmy teraz metody uzupełniania wartości brakujących:

* Jeżeli w kolumnie występują wartości wybrakowane, można je uzupełnić wskazaną wartością.

In [23]:
data[data['MedInc'].isnull()].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
25,NaN,41.0,4.495798,1.033613,317.0,2.663866,37.85,-122.28,1.07500
2415,NaN,32.0,4.784232,0.979253,761.0,3.157676,36.59,-119.44,0.67600
5243,NaN,27.0,7.737037,1.122222,654.0,2.422222,34.12,-118.42,5.00001
6031,NaN,34.0,5.528226,0.989919,1761.0,3.550403,34.08,-117.72,1.28800
6348,NaN,52.0,3.428571,0.857143,46.0,6.571429,34.06,-117.75,0.67500


In [24]:
missing_indexes = data[data['MedInc'].isnull()].index # zapamiętujemy brakujące indeksy

In [25]:
data['MedInc'] = data['MedInc'].fillna(0) # uzupełniamy wszystko wartością 0

In [26]:
data.loc[missing_indexes].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
25,0.0,41.0,4.495798,1.033613,317.0,2.663866,37.85,-122.28,1.07500
2415,0.0,32.0,4.784232,0.979253,761.0,3.157676,36.59,-119.44,0.67600
5243,0.0,27.0,7.737037,1.122222,654.0,2.422222,34.12,-118.42,5.00001
6031,0.0,34.0,5.528226,0.989919,1761.0,3.550403,34.08,-117.72,1.28800
6348,0.0,52.0,3.428571,0.857143,46.0,6.571429,34.06,-117.75,0.67500


Aby umieścić wartości wybrakowane masowo w oryginalnej ramce danych należy użyć metody *fillna* na obiekcie klasy DataFrame przekazując jako parametr słownik mapujący nazwy kolumn na wartości, którymi mają zostać zastąpione wartości wybrakowane.

In [27]:
missing_indexes = data[(data['Longitude'].isnull()) | (data['Latitude'].isnull())].index

In [28]:
data.fillna({
    'Longitude': 0,
    'Latitude': 100,
}, inplace = True)

In [29]:
data.loc[missing_indexes].head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
54,1.6098,52.0,5.021459,1.008584,701.0,3.008584,100.00,-122.28,0.875
66,0.8056,48.0,4.382530,1.066265,788.0,2.373494,37.81,0.00,0.844
136,6.8538,29.0,6.657993,1.007435,661.0,2.457249,100.00,-122.19,3.689
574,4.7695,23.0,4.933970,1.103388,2690.0,2.337098,100.00,-122.27,2.917
757,2.6719,28.0,3.884157,1.039182,1255.0,2.137990,100.00,-122.07,1.612


Biblioteka pandas dostarcza metod wyznaczających podstawowe (i bardziej zaawansowane) statystyki. Na szczególną uwagę zasługują metody *mean()* i *median()*, które zwracają odpowiednio średnią arytmetyczną i medianę wartości ze wskazanego atrybutu.

In [30]:
data['MedHouseVal'].mean()

2.0682738501843585

In [31]:
data['MedHouseVal'].median()

1.797

W przypadku wyznaczania dominanty zastosowanie znajduje metoda *mode()*. Należy jednak pamiętać, że dominantą nie zawsze musi być tylko jedna wartość.

In [32]:
data['Population'].mode()

0    891.0
Name: Population, dtype: float64

In [33]:
data['Population'].mode().iloc[0]

891.0

## Oczyszczanie danych z biblioteką Scikit-learn

Oprócz biblioteki *pandas*, biblioteka *Scikit-learn* zawiera także obszerny zestaw narzędzi do pracy z oczyszczaniem danych, w szczególności z uzupełnianiem danych wybrakowanych. Przeznaczona do tego celu klasa *SimpleImputer* przyjmuje w inicjalizatorze parametr *strategy*, w którym należy wskazać metodę uzupełniania brakujących wartości:
- mean,
- median
- most_frequent,
- constant.

In [34]:
from sklearn.impute import SimpleImputer

In [35]:
imputer = SimpleImputer(strategy='median')

Przed właściwym uzupełnieniem wartości należy najpierw wybrać zestaw pasujących atrybutów, dla których zostanie zastosowana wybrana strategia. Pamiętaj, że używamy:
- średniej arytmetycznej lub mediany (dla wartości rzeczywistych),
- dominanty (dla danych kategorialnych).

In [36]:
num_attributes = data.select_dtypes(include=['number'])
num_attributes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [37]:
num_attributes.isnull().any(axis=0)

MedInc         False
HouseAge        True
AveRooms        True
AveBedrms       True
Population      True
AveOccup        True
Latitude       False
Longitude      False
MedHouseVal     True
dtype: bool

Wywołanie metody *fit* na utworzonym obiekcie pozwoli na automatyczne wyznaczenie wartości do uzupełnienia w każdym z atrybutów.

In [38]:
imputer.fit(num_attributes)

SimpleImputer(strategy='median')

W atrybucie *statistics_* zawarte są wyznaczone wartości do zastąpenia brakujących według obranej strategii.

In [39]:
imputer.statistics_

array([ 3.53380000e+00,  2.90000000e+01,  5.22909091e+00,  1.04878049e+00,
        1.16600000e+03,  2.81829489e+00,  3.42600000e+01, -1.18490000e+02,
        1.79700000e+00])

Do zastąpienia brakujących wartości przeznaczona jest metoda *transform*.

In [40]:
missing_indexes = num_attributes[num_attributes.isnull().any(axis = 1)].index
missing_indexes

Index([  121,   137,   255,   263,   276,   279,   304,   363,   479,   517,
       ...
       19684, 19704, 19728, 19816, 20013, 20030, 20422, 20527, 20548, 20549],
      dtype='int64', length=175)

In [41]:
new_num_attributes = imputer.transform(num_attributes)

In [42]:
import pandas as pd

In [43]:
new_num_attributes = pd.DataFrame(new_num_attributes, columns=data.columns)
new_num_attributes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [44]:
pd.DataFrame(imputer.statistics_.reshape((1,9)), columns= new_num_attributes.columns)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,3.5338,29.0,5.229091,1.04878,1166.0,2.818295,34.26,-118.49,1.797


In [45]:
new_num_attributes.loc[missing_indexes]

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
121,4.9643,37.0,5.229091,1.148936,98.0,2.085106,37.85,-122.24,3.35000
137,8.3170,32.0,6.977186,1.003802,635.0,2.818295,37.82,-122.19,3.65900
255,2.3309,46.0,3.485876,1.059322,1166.0,3.341808,37.77,-122.21,0.98700
263,2.8980,43.0,5.020661,1.037190,1166.0,3.175620,37.77,-122.20,1.21400
276,3.9712,46.0,6.410334,1.176292,1166.0,2.802432,37.79,-122.18,2.08100
...,...,...,...,...,...,...,...,...,...
20030,3.2500,19.0,5.800469,1.011737,1166.0,2.441315,36.08,-119.03,0.80600
20422,5.1457,35.0,5.229091,1.217593,576.0,2.666667,34.14,-118.90,5.00001
20527,1.4653,7.0,3.525794,1.017857,1166.0,8.886905,38.54,-121.79,3.10000
20548,1.3942,38.0,5.229091,0.941799,701.0,3.708995,38.68,-121.76,0.69400


In [46]:
new_num_attributes.isnull().any(axis=0)

MedInc         False
HouseAge       False
AveRooms       False
AveBedrms      False
Population     False
AveOccup       False
Latitude       False
Longitude      False
MedHouseVal    False
dtype: bool

Z uwagi na różnice między interfejsami bibliotek *pandas* i *Scikit-learn* warto zwrócić uwagę na typowe aspekty interfejsu aktualnie używanego narzędzia. Wykorzystywane są dwie niezależne metody: *fit* oraz *transform*. Wywołanie metody fit oznacza dopasowanie do aktualnie przekazanej ramki danych i wyznaczenie na jej podstawie wartości do uzupełnienia. Analogicznie wygląda sytuacja dla przekazanego atrybutu. Wywołanie metody *transform* spowoduje faktyczne uzupełnienie brakujących wartości i zwrócenie utworzonego w ten sposób nowego obiektu będącego pełną ramką danych. Warto więc mieć na uwadze, że próba wywołania metody *transform* po wywołaniu metody *fit* na innym zbiorze danych może kompletnie mijać się z celem.

## Zadania

1. Pobrać i wczytać zbiór danych danych dostępny pod adresem: https://archive.ics.uci.edu/dataset/10/automobile

2. Wartości wybrakowane w zbiorze oznaczone są symbolem "?". W celu zamiany na wartość *None* - dopasowaną do języka Python - można wykorzystać metodę *replace* wywoływaną na obiekcie ramki *pandas*. Utworzyć głęboką kopię powstałej w ten sposób ramki danych (funkcja [deepcopy](https://docs.python.org/3/library/copy.html)).

3. Zastosować poznane metody uzupełniania wartości wybrakowanych dopasowanych do typów danych w atrybutach za pomocą biblioteki *pandas*. W przypadku gdy do danego typu pasuje więcej niż jedna strategia (np. do typu ciągłego numerycznego: mediana i średnia arytmetyczna), utworzyć dwie wersje kolumn (np. col1\_mean i col1\_median), gdzie w każdej będą wartości uzupełnione inną strategią.

4. Na podstawie utworzonej kopii ramki z punktu 2, powtórzyć kroki z punktu 3, ale przy użyciu biblioteki *Scikit-learn*.

